# Laboratório 5: Reamostragem e Bootstrapping [<img src="images/colag_logo.svg" style="float: right; margin-right: 0%; vertical-align: middle; width: 6.5%;">](https://colab.research.google.com/github/urielmoreirasilva/urielmoreirasilva.github.io/blob/main/labs/Laborat%C3%B3rio%205%2FLaborat%C3%B3rio%205.ipynbLaborat%C3%B3rio%205%2FLaborat%C3%B3rio%205.ipynb) [<img src="images/github_logo.svg" style="float: right; margin-right: 0%; vertical-align: middle; width: 3.25%;">](https://github.com/urielmoreirasilva/urielmoreirasilva.github.io/blob/main/labs/Laborat%C3%B3rio%205%2FLaborat%C3%B3rio%205.ipynb)

Bem-vindo ao Laboratório 5! Nesta tarefa, desenvolveremos uma compreensão mais aprofundada sobre estimativa de parâmetros e inicialização.

Você deve concluir todo este laboratório e enviá-lo ao Moodle até às 23h59 da data de vencimento.

### Referências
- [CIT: Capítulos 9, 10 e 13](https://inferentialthinking.com/)
- Aulas: Tópicos 11, 12, 13 e 14. 

Material adaptado do [DSC10 (UCSD)](https://dsc10.com/) por [Flavio Figueiredo (DCC-UFMG)](https://flaviovdf.io/fcd/) e [Uriel Silva (DEST-UFMG)](https://urielmoreirasilva.github.io)

In [20]:
## Imports para esse laboratório
import numpy as np
import pandas as pd
import math

## Opções do MatplotLib 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 5)

## 1. Percentis 🅿️

#### Pergunta 1.1.

Digamos que você esteja em uma turma com 10 alunos e as notas de todos os alunos da turma estão armazenadas na matriz `grades`. Sua pontuação é 84.

In [24]:
grades = np.array([56, 65, 67, 72, 74, 78, 78, 80, 84, 94])

[!]

Qual das seguintes afirmações são **verdadeiras**? Utilizando a _definição_ de percentil, preencha as respostas correspondentes na célula de Markdown seguinte.

**A**. A pontuação mais alta está no percentil 100.

**B**. Sua pontuação é superior ao percentil 80.

**C**. Sua pontuação é inferior ao percentil 81.

**D**. Sua pontuação está no percentil 86.

**E**. Uma pontuação de 78 está no percentil 50.

> ...

#### Pergunta 1.2.

Use a função `np.percentile` para calcular o percentil 50 de `grades` e salve o resultado como `p_50`.

In [39]:
p_50 = ...
p_50

Ellipsis

#### Pergunta 1.3.

Use `np.median` para calcular a mediana de `grades` e salve o resultado como `median_grade`.

In [58]:
median_grade = ...
median_grade

Ellipsis

#### Pergunta 1.4.

Compare **manualmente** as respostas das Perguntas 1.3. e 1.4.  Defina a variável `same` como `True` se os dois valores forem iguais, e `False` se forem diferentes.

_<ins> Desafio </ins>_: tente não usar if/else aqui!

In [62]:
same = ...
same

Ellipsis

## 2. Preliminares da Inteligência Aliada 🧠

Ao longo deste laboratório, estudaremos um problema estatístico conhecido como [German tank problem](https://en.wikipedia.org/wiki/German_tank_problem).

Na Segunda Guerra Mundial, os Aliados (liderados pelos EUA, Reino Unido e União Soviética) queriam saber quantos tanques militares os alemães tinham produzido. No entanto, eles não conseguiam ter acesso aos dados dessa produção – tudo o que conseguiam observar era apenas uma **amostra** aleatória de tanques capturados em batalha.

Para definir o problema de maneira mais precisa, suponha então que os tanques produzidos recebiam números de série (_serial numbers) que variavam de 1 a `N`, onde `N` era o número total de tanques produzidos. Os Aliados estavam então tentando estimar `N` (um **parâmetro populacional**) utilizando os números de série dos tanques capturados (sua amostra). 

Neste laboratório, assumiremos que a amostra dos Aliados era uma **amostra aleatória simples** da população de tanques produzidos (lembre-se: isso significa que cada elemento era amostrado de maneira independente, uniforme e _sem reposição_).

<br>

<center><img src='https://raw.githubusercontent.com/dsc-courses/dsc10-2023-wi/0c3fbe39e35e9db8cc1716ec5feb8c2f325cc103/labs/lab05/images/tank.jpg' width=400></center>

<br>

#### **Pergunta 2.1.** 

Se calcularmos uma estimativa de `N` com base na nossa amostra, essa quantidade é um _parâmetro_ ou uma _estatística_? Preencha a célula de Markdown seguinte com as afirmativas **verdadeiras**.

**A**. `N` é um parâmetro, e logo uma estimativa para `N` calculada com base na amostra também é um parâmetro.

**B**. `N` é um parâmetro, e uma estimativa para `N` calculada com base na amostra é uma estatística.

**C**. `N` é uma estatística, e uma estimativa para `N` calculada com base na amostra é um parâmetro.

**D**. `N` é uma estatística, e logo uma estimativa para `N` calculada com base na amostra também é uma estatística.

> ...

Para tornarmos o problema considerado nesse laboratório mais realista, suponha que aqui não tenhamos acesso ao número verdadeiro `N` de tanques produzidos pelos alemães, mas apenas uma amostra aleatória simples dos números de série dos tanques capturados pelos Aliados.

Esses números de série estão contidos no DataFrame `observations` abaixo.

In [108]:
observations = pd.read_csv("https://raw.githubusercontent.com/dsc-courses/dsc10-2023-wi/main/labs/lab05/data/serial_numbers.csv")
num_observations = observations.shape[0]
observations

,serial_number
0,47
1,42
2,57
3,79
4,26
5,23
6,36
7,64
8,83
9,135


#### **Pergunta 2.2.** 

Defina uma função chamada `plot_serial_numbers` que desenha um histograma de qualquer DataFrame de números de série.  Deve levar um argumento, um DataFrame `df` com uma única coluna chamada `'serial_number'` (como `observations`).  Ele deve traçar um histograma dos valores na coluna `'serial_number'` **usando bins de largura 1** variando de **1 a 200 (inclusivo)** mas não retornar nada.  Em seguida, chame essa função para fazer um histograma de `observations`.

In [ ]:
def plot_serial_numbers(df):
    ...

# Chame sua função.
...

***Verifique sua resposta***: Seu histograma deve ter barras com a mesma altura e o eixo x deve variar de 0 a 200.

##### **Pergunta 2.3.** 

Como estamos tentando estimar o máximo da população, `N`, uma estatística natural a ser usada é a amostra **max**. Em outras palavras, podemos estimar o número total de tanques como sendo o maior número de série da nossa amostra.

Abaixo, escreva uma função chamada `calculate_max_based_estimate` que calcula essa estatística em uma determinada série de números de série. Deve tomar como argumento uma série de números de série e retornar seu máximo.

Depois disso, use-o para calcular uma estimativa de `N` usando os números de série em `observações`. Chame a estimativa de `max_based_estimate`.

In [ ]:
def calculate_max_based_estimate(nums):
    ...

max_based_estimate = ...
max_based_estimate

#### **Pergunta 2.4.** 

Outra maneira de estimar `N` é tomar **duas vezes a média** dos números de série em nossa amostra. Abaixo, escreva uma função chamada `calculate_mean_based_estimate` que calcula essa estatística. Deve tomar como argumento uma série de números de série e retornar o dobro da média.

Depois disso, use-o para calcular uma estimativa de `N` usando os números de série em `observações`. Chame a estimativa de `mean_based_estimate`.

In [ ]:
def calculate_mean_based_estimate(nums):
    ...

mean_based_estimate = ...
mean_based_estimate

#### **Pergunta 2.5.** 

Observe os valores de `max_based_estimate` e `mean_based_estimate` que obtivemos para nosso conjunto de dados:

In [ ]:
max_based_estimate

In [ ]:
mean_based_estimate

O valor de `max_based_estimate` diz algo sobre `mean_based_estimate`.  Poderia nossa atual `mean_based_estimate` possivelmente ser igual a `N` (pelo menos se arredondarmos para o número inteiro mais próximo)?  Se não, é definitivamente mais alto, definitivamente mais baixo, ou não podemos dizer?  Atribua uma das opções (1-6) à variável `preliminaries_q5` abaixo.
1. Sim, nossa `mean_based_estimate` para esta amostra pode ser igual a `N`.
2. Não, nossa `mean_based_estimate` para esta amostra não pode ser igual a `N`, é definitivamente menor em aproximadamente 3.
3. Não, nossa `mean_based_estimate` para esta amostra não pode ser igual a `N`, é definitivamente menor em pelo menos 12.
4. Não, nossa `mean_based_estimate` para esta amostra não pode ser igual a `N`, é definitivamente maior em aproximadamente 3.
5. Não, nossa `mean_based_estimate` para esta amostra não pode ser igual a `N`, é definitivamente maior em pelo menos 12.
6. Não, nossa `mean_based_estimate` para esta amostra não pode ser igual a `N`, mas não podemos dizer se é menor ou maior.

In [ ]:
preliminaries_q5 = ...

Não podemos simplesmente proclamar com segurança que `max_based_estimate` ou `mean_based_estimate` são iguais a `N`, porque não sabemos o que `N` realmente é.  E se estivermos muito longe? Queremos ter uma noção da precisão de nossas estimativas.

## 3. Bootstrapping 🥾

#### **Pergunta 2.1.** 

Escreva uma função chamada `simulate_resample`.  Não deve receber argumentos e deve gerar uma nova amostra (novamente, **com substituição**) a partir dos números de série observados em `observations` e retornar essa nova amostra.  (A reamostragem deve ser um DataFrame como `observations`.)

_Dica:_ Use o método `.sample`.

In [ ]:
def simulate_resample():
    ...

# Esse é um pequeno truque para garantir a reproducibilidade dos resultados,
# isto é, garantir que, embora usemos funções aleatórias (.sample), nós obtenhamos o mesmo resultado.
np.random.seed(123)

one_resample = simulate_resample()
one_resample

Posteriormente, usaremos muitas reamostras de uma vez para ver como normalmente são as estimativas.  Muitas vezes não prestamos atenção a reamostras únicas, por isso é fácil entendê-las mal.  Vamos examinar algumas reamostras individuais antes de começarmos a usá-las.

**Pergunta 2.2.** Faça um histograma de sua `one_resample` **e** um histograma separado das observações originais. Certifique-se de usar a função `plot_serial_numbers` que você definiu anteriormente no laboratório.

In [ ]:
# Histograma da reamostragem.
...

In [ ]:
# Histogram das observações originais.
...

**Pergunta 2.3.** Quais das seguintes afirmações são verdadeiras:
1. No gráfico da reamostragem, não há barras em locais que não existiam no gráfico das observações originais.
2. No gráfico das observações originais, não há barras em locais que não existiam no gráfico da reamostragem.
3. Não há números de série duplicados na reamostra.
4. Não há números de série duplicados nas observações originais.

Atribua `true_statements` a uma lista de declarações corretas

In [ ]:
true_statements = ...

**Pergunta 2.4.** Crie mais 2 reamostras. Para cada reamostra, trace um histograma e calcule as estimativas baseadas no máximo e na média usando essa reamostra.

In [ ]:
# np.arange(2) pois queremos dois resample. Também poderíamos usar range(2)
for i in np.arange(2):
    resample = ...

    # Plote o histograma dessa reamostragem.
    ...

    # Calcule as estimativas baseadas na média e no máximo da reamostragem.
    mean_based_estimate_resample = ...
    max_based_estimate_resample = ...

    # Don't change the following 3 lines:
    print(f"Estimativa baseada na média da reamostragem {i+1}:", mean_based_estimate_resample)
    print(f"Estimativa baseada no máximo do reamostragem {i+1}:", max_based_estimate_resample)
    print('\n')

Há uma boa chance de você descobrir que as estimativas baseadas no máximo das reamostras são exatamente 135 (execute a célula algumas vezes e você quase certamente verá isso acontecer). Você provavelmente também descobrirá que as duas estimativas baseadas na média diferem da estimativa baseada na média da amostra (e uma da outra).

**Pergunta 2.5.** Calcule a probabilidade exata de que uma estimativa baseada no máximo de uma *reamostra* de nossa amostra de observações seja 135 e atribua-a à variável `resampling_q5` abaixo. Pode ser útil lembrar que o tamanho das “observações” é 17.

Observe que esta é uma questão de **matemática**, não de programação. Pode ser útil descobrir sua resposta no papel e então atribuir `resampling_q5` a uma expressão que avalie a resposta correta.

_Dica:_  Pense no exemplo da "vovó" da Aula 11. Qual é a probabilidade de qualquer um dos elementos em nossa reamostra ser igual a 135?

In [ ]:
resampling_q5 = ...
resampling_q5

A resposta correta é alta, acima de 60%. Pense por que é menos provável que uma estimativa baseada na média de uma reamostra seja exatamente igual à estimativa baseada na média da amostra original em comparação com uma estimativa baseada no máximo.

## 3. Reamostragem via Simulação 💻

Como a reamostragem de uma grande amostra aleatória se parece com a amostragem de uma população, o código também deve ser quase o mesmo. Isso significa que podemos escrever uma função que simule a amostragem de uma população ou a reamostragem de uma amostra.  Se lhe passarmos uma população como argumento, ele fará o primeiro; se passarmos uma amostra, ele fará o último.

**Questão 3.1.** Escreva uma função chamada `simulate_estimates`. Deve levar 4 argumentos:
- `original_df`: Um DataFrame do qual os dados devem ser amostrados, com 1 coluna chamada `'serial_number'`.
- `sample_size`: O tamanho de cada amostra, um número inteiro. (Por exemplo, para fazer a reamostragem, passaríamos o número de linhas em `original_df` para este argumento.)
- `statistic`: Uma **função** que calcula uma estatística em uma amostra. Este argumento é o **nome de uma função** que recebe uma série de números de série como argumento e retorna um número (por exemplo, `calculate_mean_based_estimate`).
- `repetitions`: O número de repetições a realizar (ou seja, o número de reamostras a criar).

Deve-se simular `repetitions` amostras **com substituição** do DataFrame fornecido. Para cada uma dessas amostras, deve-se calcular a estatística dessa amostra. Ento, deve-se retornar um array contendo o valor daquela estatística para cada amostra (isso significa que o comprimento do array retornado deve ser igual a `repetitions`).

O código abaixo fornece um exemplo de uso da sua função e descreve como você pode verificar se a escreveu corretamente.

***Verifique sua resposta***: O histograma que você vê deve ser uma curva em forma de sino centrada em 1000 com a maior parte de sua massa em [800, 1200].

In [ ]:
def simulate_estimates(original_df, sample_size, statistic, repetitions):
    # Nossa implementação dessa função levou cerca de 4-5 linhas de código.
    ...

# Isso deve gerar um histograma empírico com estimativas de duas vezes a média
# de N exemplos de tamanho 50 se N for 1000. .
# Observe que o argumento da estatística é calculado usando calculate_mean_based_estimate.
example_estimates = simulate_estimates(
    pd.DataFrame().assign(serial_number=np.arange(1, 1000+1)),
    50,
    calculate_mean_based_estimate,
    10000)
pd.DataFrame().assign(mean_based_estimate = example_estimates).plot(kind = 'hist', density=True, bins=np.arange(500, 1500, 25), ec='w');

Agora podemos voltar à amostra que realmente observamos (`observations`) e estimar o quanto nossa estimativa de `N` baseada na média teria variado de amostra para amostra.

**Questão 3.2.** Usando o procedimento bootstrap e as `observações` de amostra, simule a distribuição aproximada de **estimativas baseadas em médias** de `N`. Use 5.000 repetições. Armazene as estimativas em `bootstrap_estimates`. (Observe que isso requer apenas uma linha de código; chame sua função `simulate_estimates`.)

Fornecemos um código que traça um histograma, permitindo visualizar as estimativas simuladas.

In [ ]:
bootstrap_estimates = ...
pd.DataFrame().assign(mean_based_estimate = bootstrap_estimates).plot(kind = 'hist', density=True, bins=np.arange(0, 200, 4), ec='w');

**Pergunta 3.3.** Calcule um intervalo que cubra os 95% intermediários das estimativas de bootstrap.  Verifique se o seu intervalo parece cobrir 95% da área no histograma acima.


_Dicas:_
- Use [`np.percentile`](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html) aqui.
- Se você estiver usando 5 e 95 como argumentos para `np.percentile`, tente novamente – apenas 90% dos dados estão entre os percentis 5 e 95!

In [ ]:
left_end = ...
right_end = ...
print("Meio 95% das estimativas do bootstrap: [{:f}, {:f}]".format(left_end, right_end))

**Pergunta 3.4.** Digamos que `N`, o parâmetro populacional que estamos tentando estimar, seja na verdade 150. Escreva o código que simula o processo de amostragem e inicialização novamente, como segue:

1. Gere um novo conjunto de observações aleatórias que os Aliados possam ter visto por meio de amostragem do DataFrame populacional que criamos para você abaixo. Pegue uma amostra de tamanho 70 **sem reposição**. Armazene a amostra no nome da variável `new_observations`
2. Usando apenas `new_observations` – não `population` – calcule 5.000 estimativas bootstrap de `N` baseadas em média. Para fazer isso, chame sua função `simulate_estimates`.
3. Calcule um intervalo que cubra os 95% intermediários dessas estimativas baseadas na média.

In [ ]:
population = pd.DataFrame().assign(serial_number = np.arange(1, 150+1))
new_observations = ...
new_bootstrap_estimates = ...

new_left_end = ...
new_right_end = ...

print("Meio 95% das estimativas do bootstrap: [{:f}, {:f}]".format(new_left_end, new_right_end))

**Pergunta 3.5.** Se você executasse sua célula acima muitas e muitas vezes, aproximadamente qual porcentagem dos intervalos criados incluiria `N` (150 neste caso)?  Atribua 1, 2, 3, 4 ou 5 à variável `simulating_q5` abaixo.

1. 100%
2. 97.5%
3. 95%
4. 5%
5. É impossível dizer.

In [ ]:
simulating_q5 = ...

## 4. Com ou sem reposição? 🔂

Cada vez que reamostramos nossa amostra original, amostramos **com reposição**. O que aconteceria se tentássemos reamostrar sem substituição? Vamos descobrir!

Abaixo, coletaremos outra amostra aleatória de tamanho 70 da `população` da qual poderemos reamostrar. Chamaremos isso de `original_sample`.

In [ ]:
np.random.seed(23) #Para garantir que obtenhamos o mesmo resultado.
original_sample = population.sample(70)
original_sample

**Pergunta 4.1.** Abaixo, 5.000 vezes, colete uma nova amostra de tamanho 70 **de `original_sample` sem reposição**. Calcule a estimativa baseada na média em cada reamostra e armazene as estimativas na matriz `estimates_without_replacement`.

Nota: Você **não** pode usar sua função `simulate_estimates` aqui, porque ela faz amostras com substituição. Em vez disso, você terá que escrever um novo loop for. É uma boa ideia começar copiando o código da sua função no 3.1 e alterando as peças necessárias.

In [ ]:
estimates_without_replacement = ...
estimates_without_replacement

**Pergunta 4.2.** Se você completou 4.1 corretamente, notará que todas as 5.000 estimativas são idênticas e equivalem a aproximadamente 149,5143. Além disso, este número é igual à estimativa baseada na média derivada de `amostra_original`, sem qualquer reamostragem:

In [ ]:
calculate_mean_based_estimate(original_sample.get('serial_number'))

Por que todas as nossas estimativas são idênticas e por que **devemos** amostrar com reposição durante a reamostragem?

<!-- COMEÇAR PERGUNTA -->

<!--
COMEÇAR PERGUNTA
nome: q4_2
manual: verdadeiro
-->

_Digite sua resposta aqui, substituindo este texto._

<!-- FIM DA PERGUNTA -->



## Linha de chegada 🏁

Parabéns! Você concluiu o Laboratório 5.

Para enviar sua tarefa:

1. Selecione `Kernel -> Reiniciar e executar tudo` para garantir que você executou todas as células, incluindo as células de teste.
2. Leia o caderno para ter certeza de que está tudo bem e que todos os testes foram aprovados.
3. Baixe seu notebook usando `Arquivo -> Baixar como -> Notebook (.ipynb)` e, em seguida, carregue seu notebook para o Moodle.
